In [14]:
import os
from modules.extract_results import display_results_by_subdistribution, display_results_by_named_entity, \
                                    display_confusion_matrix, extract_results_to_txt_file

In [16]:
for directory in os.listdir("models"):
    if directory.startswith("model_"):
        try:
            print("================================================================")
            print("Mudel: ", directory)
            display_results_by_subdistribution(os.path.join('models', directory, 'results.txt'))
            display_results_by_named_entity(os.path.join('models', directory, 'results.txt'))
        except:
            print("results.txt faili ei eksisteeri")

Mudel:  model_default


Alamhulk,1,2,3,4,5,Total
Precision,0.911927,0.913476,0.911007,0.902855,0.904527,0.906989
Recall,0.879646,0.884364,0.883126,0.872729,0.874598,0.877350
F1-score,0.895495,0.898684,0.896850,0.887537,0.889311,0.891923


,1,2,3,4,5,Total
ORG_precision,0.777778,0.777778,0.803279,0.792829,0.777108,0.785754
ORG_recall,0.711864,0.728000,0.753846,0.742537,0.745665,0.736383
ORG_f1score,0.743363,0.752066,0.777778,0.766859,0.761062,0.760226
PER_precision,0.939442,0.942244,0.941482,0.932070,0.935040,0.938056
PER_recall,0.927235,0.927621,0.927866,0.921370,0.924899,0.925798
PER_f1score,0.933298,0.934875,0.934624,0.926689,0.929942,0.931886
MISC_precision,0.700000,0.794118,0.775701,0.770833,0.741758,0.756482
MISC_recall,0.512195,0.586957,0.619403,0.656805,0.645933,0.604258
MISC_f1score,0.591549,0.675000,0.688797,0.709265,0.690537,0.671030
LOC_precision,0.533981,0.605948,0.616967,0.601533,0.615152,0.594716


Mudel:  model_local_features_without_morph


Alamhulk,1,2,3,4,5,Total
Precision,0.858887,0.868161,0.868814,0.862795,0.864699,0.865041
Recall,0.840265,0.835786,0.835890,0.828796,0.832153,0.832976
F1-score,0.849474,0.851666,0.852034,0.845454,0.848114,0.848706


...

Alamhulk,1,2,3,4,5,Total
Precision,0.865644,0.868436,0.865858,0.855932,0.859465,0.861326
Recall,0.836726,0.836339,0.835283,0.827556,0.823870,0.829549
F1-score,0.850939,0.852086,0.850296,0.841505,0.841291,0.845139


,1,2,3,4,5,Total
ORG_precision,0.698413,0.698413,0.722513,0.711111,0.707831,0.707656
ORG_recall,0.745763,0.704000,0.707692,0.716418,0.679191,0.710613
ORG_f1score,0.721311,0.701195,0.715026,0.713755,0.693215,0.708901
PER_precision,0.912788,0.911627,0.911594,0.900183,0.902527,0.907744
PER_recall,0.886694,0.886541,0.889238,0.883209,0.884381,0.886013
PER_f1score,0.899552,0.898909,0.900277,0.891615,0.893362,0.896743
MISC_precision,0.647059,0.739726,0.742857,0.720280,0.731844,0.716353
MISC_recall,0.536585,0.586957,0.582090,0.609467,0.626794,0.588379
MISC_f1score,0.586667,0.654545,0.652720,0.660256,0.675258,0.645889
LOC_precision,0.350877,0.465704,0.479167,0.475442,0.488152,0.451868


Mudel:  model_morph_with_lemmas_and_sentences_and_gazzetteer


Alamhulk,1,2,3,4,5,Total
Precision,0.855575,0.858973,0.853547,0.846904,0.853236,0.852496
Recall,0.845354,0.845856,0.849787,0.833163,0.835586,0.839655
F1-score,0.850434,0.852364,0.851663,0.839977,0.844319,0.846027


,1,2,3,4,5,Total
ORG_precision,0.596774,0.612403,0.606796,0.606383,0.635854,0.611642
ORG_recall,0.627119,0.632000,0.641026,0.638060,0.656069,0.638855
ORG_f1score,0.611570,0.622047,0.623441,0.621818,0.645804,0.624936
PER_precision,0.900367,0.905347,0.903447,0.896281,0.902805,0.901649
PER_recall,0.892412,0.894366,0.899389,0.885646,0.892413,0.892845
PER_f1score,0.896372,0.899823,0.901414,0.890932,0.897579,0.897224
MISC_precision,0.794118,0.786667,0.743590,0.719745,0.737113,0.756247
MISC_recall,0.658537,0.641304,0.649254,0.668639,0.684211,0.660389
MISC_f1score,0.720000,0.706587,0.693227,0.693252,0.709677,0.704549
LOC_precision,0.417266,0.452145,0.470716,0.471637,0.480570,0.458467
